In [ ]:
!pip install tavily-python -qq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 14.0 MB/s eta 0:00:00


In [ ]:
import requests
from tavily import TavilyClient

def search_tavily(query, api_key):
    client = TavilyClient(api_key=api_key)

    contents = []
    data = []

    try:
        response = client.search(query=query, num_results=10)

        for result in response['results']:
            content = result['content']
            contents.append(content)

        for content in contents:
            for i in range(0, len(content), 500):
                chunk = content[i:i+500]
                data.append(chunk)

    except Exception as e:
        print(f"Error querying '{query}': {e}")

    return data


In [ ]:
content=search_tavily("Manmohan Singh's decisions that shaped a billion lives",os.environ["TAVILY_API"])

In [ ]:
import os
import google.generativeai as genai

genai.configure(api_key=os.environ["GEMINI_API_KEY"])

generation_config = {
  "temperature": 1,
  "top_p": 0.95,
  "top_k": 40,
  "max_output_tokens": 8192,
  "response_mime_type": "text/plain",
}

model = genai.GenerativeModel(
  model_name="gemini-1.5-flash",
  generation_config=generation_config,
)

def check_article_for_false_info(article_content, tavily_results):
    reliable_sources = "\n".join(tavily_results)
    combined_content = f"Article Content:\n{article_content}\n\nReliable Sources:\n{reliable_sources}"

    chat_session = model.start_chat(
        history=[]
    )

    response = chat_session.send_message(
      f"Please analyze the following article and compare its content with reliable, verified sources. Determine if the article contains false information. Provide a response of either 'True' if the article is based on accurate facts or 'False' if the article contains misinformation.\n\nArticle:\n{combined_content}\n\nResponse:"
    )


    return response.text

article = """Manmohan Singh's tenure as Prime Minister was marked by decisions that harmed India's economy. His introduction of GST increased prices, making life harder for ordinary citizens. His economic liberalization, which opened markets to foreign companies, led to job losses and exploitation. The privatization of state-owned enterprises made services more expensive and less accessible, benefiting only the rich. Singh's policies deepened inequality and left India vulnerable to financial crises.
"""


response_text = check_article_for_false_info(article, content)
print(response_text)


False

